In [33]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('../mystyle.mplstyle')

import sys
import os

# Get the current directory
current_dir = os.getcwd()

# Go one level up to the parent directory
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add the parent directory to the system path
sys.path.append(parent_dir)

# Now you can import the module
from pyfiles.image_tools import trim_whitespace

`````{admonition} Remark
:class: important

Please be aware that these lecture notes are accessible online in an '**early access**' format. They are actively being developed, and certain sections will be further enriched to provide a comprehensive understanding of the subject matter.

`````

# Understanding Coordinate Systems

## Spatial Reference Systems (SRS) and Datums

A Spatial Reference System (SRS) provides a framework to accurately determine and describe locations on the Earth's surface. It integrates a coordinate system with a datum, and often a map projection, to enable precise georeferencing of spatial data. By standardizing how locations are measured and represented, SRSs facilitate the integration and comparison of spatial data from various sources, making them indispensable in geographic information systems (GIS) {cite}`jones2014geographical, pászto2019spationomy, lu2014geodesy`.

## Components of SRS

1. **Coordinate Systems**: A coordinate system is a mathematical framework that provides a standardized method for specifying the precise location of points on the Earth’s surface. Coordinates can be expressed in various forms, most commonly as latitude and longitude in a geographic coordinate system, or as Cartesian coordinates $(x, y, z)$ in a projected coordinate system. The choice of coordinate system affects how spatial data is interpreted and how accurate the representation of location is in different contexts.

2. **Datums**: A datum serves as the foundational reference point that defines the origin, orientation, and scale of a coordinate system. It ensures that the coordinates correspond to actual physical locations on Earth by aligning the coordinate system with an ellipsoid that approximates the Earth's shape. The datum accounts for variations in the Earth's surface by establishing a reference ellipsoid and anchoring it at a specific location, known as the datum origin. This alignment is crucial for maintaining spatial accuracy, especially when integrating data from different sources or conducting geospatial analyses.

3. **Map Projections**: A map projection is a method used to represent the curved surface of the Earth on a flat plane, such as a map or a digital screen. This process involves mathematical transformations that project the three-dimensional geographic data onto a two-dimensional surface. Different map projections are designed to preserve certain spatial properties—such as area, shape, distance, or direction—while inevitably distorting others. The choice of projection depends on the specific needs of the project, such as the scale of the area being mapped or the particular spatial properties that need to be preserved.

## Types of Coordinate Systems in GIS

GIS rely on coordinate systems to accurately represent locations on the Earth's surface. Coordinate systems are essential for mapping, spatial analysis, and data visualization. They provide a framework for defining real-world locations in digital environments.

There are several types of coordinate systems used in GIS, each serving different purposes and suitable for various applications. The three main types are:

- Geographic Coordinate Systems (GCS)
- Projected Coordinate Systems (PCS)
- Local Coordinate Systems

## Geographic Coordinate Systems (GCS)

Geographic Coordinate Systems are based on a three-dimensional model of the Earth, which is either an ellipsoid or a sphere. Locations on the Earth's surface are defined using angular measurements, typically in decimal degrees. These measurements include degrees of longitude (x-coordinates) and degrees of latitude (y-coordinates) {cite}`kumar2023geographic`. 


````{prf:example}
:label: example2.3.1

The coordinates of Calgary, Alberta, Canada are approximately 51.0447° N latitude and 114.0719° W longitude. These coordinates pinpoint Calgary's location on the globe with high precision. The World Geodetic System 1984 (WGS84) is a widely used GCS, especially in GPS technology, providing a standard reference for mapping and navigation.


{numref}`Calgary_wgs84` shows a world map in the Mercator projection, pinpointing the location of Calgary, Alberta, Canada. The coordinates given are **51.0474°N latitude** and **-114.0719°W longitude**, which are in the WGS84 format commonly used for GPS.

- **Latitude (51.0474°N)**: Indicates Calgary is 51.0474 degrees north of the equator.
- **Longitude (-114.0719°W)**: Indicates Calgary is 114.0719 degrees west of the Prime Meridian.

```{figure} Calgary_wgs84.png
---
align: center
name: Calgary_wgs84
---
Calgary, Alberta (WGS84 Coordinates).
```
````

In [34]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import geopandas as gpd

# Calgary's coordinates (WGS84)
calgary_lat = 51.0447
calgary_lon = -114.0719

# Create a new figure and axis
fig = plt.figure(figsize=(11, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Add features
ax.set_global()
ax.stock_img()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='--')
ax.add_feature(cfeature.LAKES)
ax.add_feature(cfeature.RIVERS)

# Add gridlines
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Plot and annotate Calgary
ax.plot(calgary_lon, calgary_lat, 'ro', markersize=5, transform=ccrs.Geodetic())
ax.text(calgary_lon, calgary_lat + 3, f'Calgary\n({calgary_lat:.4f}°N, {calgary_lon:.4f}°W)', 
        color='red', fontweight='normal', fontsize=12, transform=ccrs.Geodetic(),
        va='bottom', ha='center', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

# Set title
# plt.title("Calgary, Alberta (WGS84 Coordinates)", fontsize=16)
plt.tight_layout()
fig.savefig('Calgary_wgs84.png', bbox_inches='tight', dpi=300)
plt.close()

## Projected Coordinate Systems (PCS)

Projected Coordinate Systems (PCS) are used to represent the curved surface of the Earth on a flat plane, such as a map. Unlike Geographic Coordinate Systems (GCS), which use angular measurements (latitude and longitude), PCS use linear measurements (x and y coordinates). This makes it easier to measure distances and areas accurately on a flat map {cite}`kumar2023geographic`.

A PCS is composed of two main components: a GCS and a map projection. The GCS provides the framework of latitude and longitude coordinates, while the map projection involves mathematical calculations to convert these angular coordinates into Cartesian coordinates (x, y). This conversion process allows for the creation of flat maps from the Earth's curved surface.

One common example of a PCS is the [Universal Transverse Mercator (UTM) system](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system). The UTM system divides the world into a series of zones, each 6 degrees of longitude wide. Each zone has its own PCS to minimize distortion within that zone.

For instance, Calgary, Alberta, is located in both UTM Zone 11 and Zone 12 ([See this link](https://mangomap.com/robertyoung/maps/69585/what-utm-zone-am-i-in-)). By using separate zones, the UTM system ensures that maps are more accurate for regional mapping and large-scale projects.

````{prf:example}
:label: example2.3.2

Imagine you are working on a construction project in Calgary, Alberta. Using the UTM system, you would refer to Zones 11 and 12. The coordinates for a specific location in Calgary might be given as 11U 706000mE 5660000mN. This means the location is 706,000 meters east and 5,660,000 meters north within Zone 11. This precise measurement is crucial for detailed engineering work, ensuring that all elements of the project are accurately placed.

**Breakdown of UTM Coordinates:**

1. **Zone Designation (11U)**:
    - **Zone Number**: 11
        - UTM zones are divided longitudinally into 6-degree wide strips. Zone 11 covers longitudes from 114°W to 120°W, which includes part of Calgary.
    - **Latitude Band**: U
        - The "U" indicates the latitude band. UTM zones are split into bands from C to X (excluding I and O), with "U" covering latitudes from 48°N to 56°N.

2. **Easting (706000mE)**:
    - This value represents the distance in meters east from the central meridian of the UTM zone.
    - The central meridian of Zone 11 is at 117°W. A false easting of 500,000 meters is applied to ensure all eastings within the zone are positive.
    - Thus, 706,000 meters east means the location is 206,000 meters east of the central meridian.

3. **Northing (5660000mN)**:
    - This value represents the distance in meters north from the equator.
    - For the Northern Hemisphere, northing values start at 0 at the equator and increase northward.
    - Hence, 5,660,000 meters north indicates the distance from the equator to the location in Calgary.

{numref}`utm_example_calgary` illustrates the Universal Transverse Mercator (UTM) zones, which are used for mapping and navigation. The world is divided into 60 vertical zones, each 6 degrees of longitude wide, labeled from 1 to 60. These zones are further divided into horizontal bands, labeled from C to X (excluding I and O), each 8 degrees of latitude high.

- **Zone Numbers**: The numbers at the bottom of the map (1 to 60) represent the UTM zones, each covering a 6-degree longitudinal strip.
- **Latitude Bands**: The letters on the side (C to X) represent 8-degree latitude bands, starting from 80° South to 84° North.
- **Highlighted Area**: The map highlights Zones 11U and 12U, which include Calgary, Alberta, showing its specific location within the global UTM grid.

```{figure} utm_example_calgary.png
---
align: center
name: utm_example_calgary
---
UTM Zones Across the World.
```

Let's zoom in in {numref}`utm_example_calgary`. {numref}`utm_example_calgary_zoom` illustrates the Universal Transverse Mercator (UTM) zones that cover Calgary, Alberta. The map highlights UTM Zone Designator "U" and UTM Zone Numbers 11 and 12.

```{figure} utm_example_calgary_zoom.png
---
width: 550px
align: center
name: utm_example_calgary_zoom
---
UTM Zones Covering Calgary, Alberta.
```
````

`````{admonition} Note
:class: warning

The letters "I" and "O" are excluded from the UTM zone designations to avoid confusion with the numbers "1" and "0"³. This helps ensure clarity and reduces the risk of errors when reading and interpreting the coordinates.

`````

In [35]:
# utm_example_calgary
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

# Create a global map with PlateCarree projection
fig = plt.figure(figsize=(11, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Plot Calgary on the globe
latitude_deg = 51.0447
longitude_deg = -114.0719
ax.plot(longitude_deg, latitude_deg, marker='o', color='blue', markersize=3, transform=ccrs.Geodetic())
ax.text(longitude_deg, latitude_deg, 'Calgary', color='blue', transform=ccrs.Geodetic())

# Set the extent to show the entire world
ax.set_global()

# Add coastlines and borders
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)

# Highlight UTM Zone 11N for reference (covering parts of Western North America)
ax.axvspan(-114, -120, color='green', linestyle='-', linewidth=1, alpha=0.3)  # Western boundary of Zone 11N
ax.axvspan(-114+6, -120+6, color='green', linestyle='-', linewidth=1, alpha=0.5)  # Western boundary of Zone 11N
ax.axhspan(48, 56, color='red', linestyle='-', linewidth=1, alpha=0.3)  # Highlight a latitude band

# Add UTM latitude bands along the right axis
utm_lat_bands = "CDEFGHJKLMNPQRSTUVWX"  # UTM latitude bands (from 80°S to 84°N)
lat_band_center = [-80 + i * 8 for i in range(len(utm_lat_bands))]

utm_lon = list(range(-180, 180, 6))
zone_number = [(lon + 180) // 6 + 1 for lon in utm_lon]

gl = ax.gridlines(xlocs= utm_lon, ylocs = lat_band_center, color='gray', linestyle='--')

# Adjust the x and y labels: Shift x labels to the right and y labels slightly above
xtick_labels = [str(zone) if zone % 2 != 0 else '' for zone in zone_number]
# Calculate the average of every two consecutive numbers
avg_utm_lon = [(utm_lon[i] + utm_lon[i + 1]) / 2 for i in range(len(utm_lon) - 1)]
ax.set_xticks(avg_utm_lon)
ax.set_xticklabels(xtick_labels[:-1], fontsize=14)
ax.set_xlabel('UTM Zone Numbers')

ytick_labels = list(utm_lat_bands)
# Calculate the average of every two consecutive numbers
lat_band_center = lat_band_center + [ax.get_ylim()[-1]]
avg_lat_band_center = [(lat_band_center[i] + lat_band_center[i + 1]) / 2 for i in range(len(lat_band_center) - 1)]
ax.set_yticks(avg_lat_band_center, crs=ccrs.PlateCarree())
ax.set_yticklabels(ytick_labels, fontsize=14)
ax.set_ylabel('UTM Zone Designators')
ax.grid(False)

# Add gridlines with degrees on top
gridlines = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
gridlines.top_labels = True
gridlines.left_labels = False
gridlines.bottom_labels = False
gridlines.right_labels = True
gridlines.xlines = False
gridlines.ylines = False

# Set title
# plt.title("UTM Zones Across the World")

plt.tight_layout()
fig.savefig('utm_example_calgary.png', bbox_inches='tight', dpi=300)
plt.close()

In [36]:
# utm_example_calgary_zoom
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import geopandas as gpd

# Create a global map with PlateCarree projection
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Plot Calgary on the globe
latitude_deg = 51.0447
longitude_deg = -114.0719

# Read the city boundary from the GeoPackage file
calgary_boundary = gpd.read_file('../data/City Boundary_20240620.gpkg')
calgary_boundary = calgary_boundary.to_crs(epsg=4326)  # Project to WGS84

# Plot the city boundary
calgary_boundary.plot(ax=ax, edgecolor='DarkGreen', facecolor='green', alpha = 0.6, linewidth=1, transform=ccrs.Geodetic())

# Set the extent based on the city boundary
minx, miny, maxx, maxy = calgary_boundary.total_bounds
ax.set_extent([-120, -108, 48, 56], crs=ccrs.PlateCarree())

# ax.plot(longitude_deg, latitude_deg, marker='o', color='blue', markersize=3, transform=ccrs.Geodetic())
ax.text(longitude_deg+.25, latitude_deg, 'Calgary', color='blue', transform=ccrs.Geodetic(), fontsize = 14)

# Set the extent to show the entire world
# ax.set_global()

# Add coastlines and borders
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
# ax.add_feature(cfeature.BORDERS)

xlocs = list(range(-120, -107, 6))
ylocs = [48, 56]
gl = ax.gridlines(xlocs= xlocs, ylocs = ylocs, color='k', linestyle='--', linewidth= 1)

ax.set_xticks([(xlocs[0] + xlocs[1])/2, (xlocs[1] + xlocs[2])/2])
ax.set_xticklabels([11,12], fontsize=14)
ax.grid(False)
ax.set_xlabel('UTM Zone Numbers = 11, 12')

# ytick_labels = list(utm_lat_bands)
ax.set_yticks([(ylocs[0] + ylocs[1])/2])
ax.set_yticklabels(['U'], fontsize=14)
ax.set_ylabel('UTM Zone Designator = U')

# Add gridlines with degrees on top
gridlines = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
gridlines.top_labels = True
gridlines.left_labels = False
gridlines.bottom_labels = False
gridlines.right_labels = True
gridlines.xlines = False
gridlines.ylines = False

# Set title
# plt.title("UTM Zones Across the World")

plt.tight_layout()
fig.savefig('utm_example_calgary_zoom.png', bbox_inches='tight', dpi=300)
plt.close()

## Local Coordinate Systems

Local Coordinate Systems are often used for engineering projects or other applications requiring high precision over a smaller area. These systems are tailored to specific locations and may not be compatible with global or regional coordinate systems. For example, a construction project might use a local coordinate system to ensure precise measurements and alignment. These systems are crucial for tasks that demand high accuracy, such as building infrastructure or conducting detailed land surveys {cite}`kumar2023geographic`.


````{prf:example} Examples of Local Coordinate Systems in Calgary, Alberta
:label: example2.3.3

1. **East Village Development**:
   - A local coordinate system is established for the East Village area.
   - The origin (0,0) is set at a reference point, possibly the Calgary City Hall.
   - A specific point of interest, such as a new building corner, is located at coordinates (1319, -326), meaning 1319 meters east and 326 meters south of the origin.

2. **Calgary Tower Area**:
   - For projects around the Calgary Tower, a local coordinate system is used.
   - The origin (0,0) might be set at a convenient reference point in the downtown area.
   - The Calgary Tower itself is located at coordinates (559, -468) in this system, indicating it is 559 meters east and 468 meters south of the chosen origin.

3. **Fort Calgary Historical Site**:
   - A separate local coordinate system is established for archaeological and restoration work at Fort Calgary.
   - The origin (0,0) could be set at a significant point within the fort's grounds.
   - A point of interest, perhaps a newly discovered artifact or a restoration project, is located at coordinates (1617, -58), meaning 1617 meters east and 58 meters south of the origin.

{numref}`local_coordinate_calgary` illustrates:
- **Calgary Local Coordinate System** (top): A local coordinate system is established with the origin set at Calgary City Hall (0,0). Several points of interest, such as East Village, Calgary Tower, and Fort Calgary, are plotted with their respective local coordinates relative to the origin. The distances are in meters, indicating the eastward and northward (or southward) positions from City Hall.

- **Calgary in 2D Map** (bottom): A cartographic 2D map of the Calgary area centered around City Hall. Major landmarks are plotted, including the same points from the local coordinate system, overlaid on a map background retrieved from OpenStreetMap. The geographic coordinates (latitude and longitude) are displayed here.

```{figure} local_coordinate_calgary.png
---
width: 700px
align: center
name: local_coordinate_calgary
---
Comparison of Local Coordinate Systems and 2D Cartographic Map in Calgary, Alberta.
```

These local coordinate systems allow for precise measurements and planning within each specific area, tailored to the needs of different projects in Calgary.
````

In [45]:
# # local_coordinate_calgary
# import matplotlib.pyplot as plt
# from pyproj import Proj, transform

# # Define the local origin (let's use Calgary City Hall as our local origin)
# origin_lat, origin_lon = 51.0486, -114.0708

# # Define the projection (UTM zone 11N for Calgary)
# proj = Proj(proj='utm', zone= 11, ellps='WGS84', preserve_units=False)

# # Convert origin to UTM coordinates
# origin_x, origin_y = proj(origin_lon, origin_lat)

# # Function to convert lat/lon to local coordinates
# def to_local_coords(lat, lon):
#     x, y = proj(lon, lat)
#     return x - origin_x, y - origin_y

# # Actual coordinates of points
# points = {
#     "East Village": (51.0452, -114.0522),
#     "Calgary Tower": (51.0442, -114.0631),
#     "Fort Calgary": (51.0475, -114.0478)
#     # "Fish Creek Park": (50.9228, -114.0617),
# }

# fig, ax = plt.subplots(figsize=(8, 8))

# # Plot the local origin
# ax.plot(0, 0, 'bo', markersize=8)
# ax.text(0, 0, 'Local Origin\n(City Hall)', fontsize=14, va='bottom', ha='center')

# # Plot each point
# for name, (lat, lon) in points.items():
#     x, y = to_local_coords(lat, lon)
#     ax.plot(x, y, 'g^', markersize=8)
#     ax.text(x, y, f"{name}\n({x:.0f}, {y:.0f})", fontsize = 14, va='bottom', ha='left')
#     print(f"{name}: ({x:.0f}, {y:.0f})")

# # Set limits and grid for local coordinates
# ax.set_xlim(-500, 2000)
# ax.set_ylim(-1000, 1000)

# # Add gridlines for better readability
# ax.grid(True)

# # Set axis labels
# ax.set_xlabel('X (meters east of origin)')
# ax.set_ylabel('Y (meters north of origin)')
# ax.set_aspect('equal')

# # Set title
# # plt.title("Calgary Local Coordinate System (Origin at City Hall)", fontsize=16)
# plt.tight_layout()
# fig.savefig('local_coordinate_calgary.png', bbox_inches='tight', dpi=300)
# plt.close()

import matplotlib.pyplot as plt
from pyproj import Proj
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyproj
import cartopy.io.img_tiles as cimgt
import matplotlib.gridspec as gridspec

# Define the local origin (Calgary City Hall)
origin_lat, origin_lon = 51.0486, -114.0708

# Define the projection (UTM zone 11N for Calgary)
proj = Proj(proj='utm', zone=11, ellps='WGS84', preserve_units=False)

# Convert origin to UTM coordinates
origin_x, origin_y = proj(origin_lon, origin_lat)

# Function to convert lat/lon to local coordinates
def to_local_coords(lat, lon):
    x, y = proj(lon, lat)
    return x - origin_x, y - origin_y

# Actual coordinates of points
points = {
    "East Village": (51.0452, -114.0522),
    "Calgary Tower": (51.0442, -114.0631),
    "Fort Calgary": (51.0475, -114.0478)
}

# Create a figure with a GridSpec layout to handle the subplots properly
fig = plt.figure(figsize=(9, 13))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1.5])  # Adjust the height ratios as needed

# Plot 1: Local Coordinate System (top)
ax1 = fig.add_subplot(gs[0])
ax1.plot(0, 0, 'bo', markersize=10)
ax1.text(0, 15, 'Local Origin\n(City Hall)', va='bottom', ha='center')

for name, (lat, lon) in points.items():
    x, y = to_local_coords(lat, lon)
    ax1.plot(x, y, 'r^', markersize=10)
    ax1.text(x, y, f"{name}\n({x:.0f}, {y:.0f})", va='bottom', ha='left')

ax1.set_xlim(-500, 2000)
ax1.set_ylim(-1000, 1000)
ax1.grid(True)
ax1.set_xlabel('X (meters east of origin)')
ax1.set_ylabel('Y (meters north of origin)')
ax1.set_aspect('equal')
# Update title for the first plot (Local Coordinate System)
ax1.set_title("Examples of Local Coordinate Systems in Calgary")

# Plot 2: Cartopy 2D Map (bottom)
# Define the EPSG:3401 projection
proj_epsg_3401 = ccrs.Projection(pyproj.CRS.from_epsg(3401))

ax2 = fig.add_subplot(gs[1], projection=proj_epsg_3401)
ax2.set_extent([-114.08, -114.04, 51.04, 51.06], crs=ccrs.PlateCarree())

ax2.add_feature(cfeature.LAND, facecolor='lightgray')
ax2.add_feature(cfeature.OCEAN)
ax2.add_feature(cfeature.COASTLINE)
ax2.add_feature(cfeature.BORDERS, linestyle=':')
ax2.add_feature(cfeature.LAKES, alpha=0.5)
ax2.add_feature(cfeature.RIVERS)

# Add OpenStreetMap as a background
osm_tiles = cimgt.OSM()
ax2.add_image(osm_tiles, 14)  # Zoom level 14

ax2.plot(origin_lon, origin_lat, 'bo', transform=ccrs.PlateCarree())
ax2.text(origin_lon, origin_lat, 'City Hall', transform=ccrs.PlateCarree(), fontsize=12, va='bottom', ha='right')

for name, (lat, lon) in points.items():
    ax2.plot(lon, lat, 'r^', transform=ccrs.PlateCarree())
    ax2.text(lon, lat, name, transform=ccrs.PlateCarree(), va='bottom', ha='left')

gl = ax2.gridlines(draw_labels=True, linestyle='--', color='gray', alpha=0.5)
gl.top_labels = False
gl.right_labels = False
# Update title for the second plot (Cartopy 2D Map)
ax2.set_title("Calgary City Hall and Surrounding Areas in 2D Map")

plt.tight_layout()
fig.savefig('local_coordinate_calgary.png', bbox_inches='tight', dpi=300)
plt.close()

## Choosing the Right Coordinate System

Selecting an appropriate coordinate system is crucial for the accuracy and usability of spatial data. The choice depends on the scale of the project, the region of interest, and the intended use of the data. 

- For global studies, a Geographic Coordinate System (GCS) like WGS84 is often used to ensure consistency across large areas. 
- For regional or local projects, a Projected Coordinate System (PCS) that minimizes distortion in that specific area is preferred. 

For example, a city map of Calgary might use the UTM Zone 11N projection (a type of PCS) to provide accurate representations of streets and buildings. This system is well-suited for Calgary as it minimizes distortion in this region and uses meters as units, which is convenient for urban planning and construction.

## Horizontal and Vertical Coordinate Systems

- **Horizontal Coordinate Systems**: These systems define where features are located across the globe. They use latitude and longitude to pinpoint locations. For example, the coordinates 51.0447° N, 114.0719° W identify downtown Calgary on a map.

- **Vertical Coordinate Systems**: These systems define how high or deep features are relative to a surface, such as sea level. For instance, Calgary's elevation is approximately 1,045 meters above sea level. Vertical coordinate systems are essential for applications like flood modeling in the Bow River basin, where knowing the elevation of land surfaces is crucial for predicting potential flood impacts.

## Importance of Coordinate Systems in GIS

Coordinate systems are vital for various GIS operations, including data integration, map creation, and spatial analysis. They ensure that spatial data from different sources can be compared and analyzed consistently. 

- **Data Integration**: When combining datasets from different sources, such as provincial land use data with municipal infrastructure data in Calgary, a common coordinate system ensures they align correctly.

- **Map Creation**: Accurate maps depend on the correct use of coordinate systems to represent spatial relationships. For example, creating a map of Calgary's public transit system requires precise positioning of bus stops, train stations, and route lines.

- **Spatial Analysis**: In spatial analysis, such as calculating distances between neighborhoods or areas of parks in Calgary, consistent coordinate systems produce reliable results.

Without a common coordinate system, it would be challenging to overlay maps (e.g., combining geological and urban development maps of Calgary), compare spatial data, or conduct accurate spatial analyses, leading to errors and misinterpretations in urban planning and development projects.